In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load Rents data
Rents = pd.read_csv('RentsTransactions/rents.csv', delimiter=';', low_memory=False)

In [ ]:
# Load Transactions Data
Transactions = pd.read_csv('RentsTransactions/transactions.csv', delimiter=';', low_memory=False)

In [ ]:
Transactions.info()

In [ ]:
Transactions['Transaction Type'].unique()

In [ ]:
Transactions = Transactions[Transactions['Transaction Type'] == 'Sales']

In [ ]:
Transactions.info()

In [ ]:
Rents['Property Size (sq.m)'] = pd.to_numeric(Rents['Property Size (sq.m)'], errors='coerce')

In [ ]:
Rents['Parking'] = pd.to_numeric(Rents['Parking'], errors='coerce')

In [ ]:
Transactions['Property Size (sq.m)'] = pd.to_numeric(Transactions['Property Size (sq.m)'], errors='coerce')

In [ ]:
Transactions['Parking'] = pd.to_numeric(Transactions['Parking'], errors='coerce')

In [ ]:
# Shared columns with high similarity to include in the merge
merge_keys = [
    'Property ID', 'Is Free Hold?', 'Nearest Metro', 
    'Nearest Mall', 'Nearest Landmark', 
    'Usage', 'Area', 'Property Type', 'Property Sub Type', 'Property Size (sq.m)', 'Parking', 'Master Project', 'Project'
]

# Perform the merge using these keys
merged_data = pd.merge(
    Rents, Transactions,
    on=merge_keys,  # Merge on the identified keys
    how='outer',     # Retain all rows from Rents
    suffixes=('', '_Transactions')  # Add suffix for Transactions columns
)

# Check the merged data
print("Merged Data Information:")
print(merged_data.info())

print("\nPreview of Merged Data:")
print(merged_data.head())


In [ ]:
# Convert numeric columns stored as strings
merged_data['Annual Amount'] = pd.to_numeric(merged_data['Annual Amount'], errors='coerce')
merged_data['Contract Amount'] = pd.to_numeric(merged_data['Contract Amount'], errors='coerce')
merged_data['Amount'] = pd.to_numeric(merged_data['Amount'], errors='coerce')

In [ ]:
# Calculate price per square meter
merged_data['Price_per_sqm'] = merged_data['Amount'] / merged_data['Property Size (sq.m)']


In [ ]:
# Compare with Market Trends

# Group by Property Type and Location to calculate median price per square meter
median_price_per_sqm = merged_data.groupby(['Property Type', 'Area'])['Price_per_sqm'].median().reset_index()

# Merge the median prices back to the main dataframe
merged_data = merged_data.merge(median_price_per_sqm, on=['Property Type', 'Area'], suffixes=('', '_median'))

# Filter properties where the price per square meter is below the median (considering a threshold)
threshold = 0.8  # Adjust this threshold to define "significantly lower"
merged_data['Below_Median'] = merged_data['Price_per_sqm'] < (merged_data['Price_per_sqm_median'] * threshold)


In [ ]:
# Calculate rental yield
merged_data['Rental_Yield'] = (merged_data['Annual Amount'] / merged_data['Amount']) * 100


In [ ]:
# Calculate average rental yield for the dataset
average_rental_yield = merged_data['Rental_Yield'].mean()

# Filter properties with higher than average rental yield
merged_data['Above_Avg_Yield'] = merged_data['Rental_Yield'] > average_rental_yield


In [ ]:
# Filter properties meeting both criteria
investment_properties = merged_data[(merged_data['Below_Median']) & (merged_data['Above_Avg_Yield'])]


In [ ]:
investment_properties.info()

In [ ]:
investment_properties.head(20)

In [ ]:
investment_properties['Area'].unique()

In [ ]:
investment_properties['Property Size (sq.m)'].unique()